In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import acquire
import prepare

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
rawdf = acquire.get_zillow_data()

In [4]:
rawdf.shape

(52441, 11)

In [5]:
rawdf.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,tax_amount
0,"-118,740133","34,251502",4.00,2.00,"1,323.00","6,825.00",NaN,06111,53.00,"453,000.00","5,235.32"
1,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",NaN,06037,40.00,"298,371.00","3,714.58"
2,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",NaN,06037,67.00,"161,897.00","2,031.39"
3,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22.00,"472,384.00","7,234.74"
4,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",NaN,06037,77.00,"58,693.00",751.66


In [6]:
df = prepare.prep_zillow(rawdf)

In [7]:
df.shape

(45603, 23)

In [8]:
df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,740133","34,251502",4.00,2.00,"1,323.00","6,825.00",0.00,06111,53,"453,000.00",...,small,0,0,1,0,1,0,1,0,0
1,"-118,217048","33,798657",4.00,2.00,"1,339.00","6,107.00",0.00,06037,40,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
2,"-118,531253","34,039467",2.00,2.00,"1,231.00","11,837.00",0.00,06037,67,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
3,"-117,613897","33,663005",5.00,3.00,"3,008.00","6,760.00",1.00,06059,22,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
4,"-118,393309","34,227422",2.00,1.00,996.00,"7,518.00",0.00,06037,77,"58,693.00",...,small,1,0,0,1,0,0,1,0,0


## Simple Model
$$ y = mx + b $$

$$y = f(x)$$

Tested out all scalers, and MinMaxScaler performs the best

In [9]:
scaled_df = df.copy()

In [10]:
scaler = MinMaxScaler()

In [9]:
# scaler = sklearn.preprocessing.RobustScaler()

In [9]:
# # Quantile scaler
# qt = sklearn.preprocessing.QuantileTransformer(n_quantiles=10, output_distribution='normal', random_state=0)

In [10]:
# qt.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

QuantileTransformer(n_quantiles=10, output_distribution='normal',
                    random_state=0)

In [11]:
# scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = qt.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [11]:
scaler.fit(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

MinMaxScaler()

In [12]:
scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']] = scaler.transform(scaled_df[['square_feet', 'lot_size', 'age', 'bedrooms', 'bathrooms']])

In [13]:
scaled_df.head()

,longitude,latitude,bedrooms,bathrooms,square_feet,lot_size,has_pool,fips_code,age,assessed_value,...,bathrooms_size,county_Los Angeles,county_Orange,county_Ventura,bedrooms_size_small,bedrooms_size_medium,bedrooms_size_large,bathrooms_size_small,bathrooms_size_medium,bathrooms_size_large
0,"-118,740133","34,251502",0.60,0.18,0.13,0.06,0.00,06111,0.38,"453,000.00",...,small,0,0,1,0,1,0,1,0,0
1,"-118,217048","33,798657",0.60,0.18,0.13,0.05,0.00,06037,0.28,"298,371.00",...,small,1,0,0,0,1,0,1,0,0
2,"-118,531253","34,039467",0.20,0.18,0.12,0.11,0.00,06037,0.48,"161,897.00",...,small,1,0,0,1,0,0,1,0,0
3,"-117,613897","33,663005",0.80,0.36,0.40,0.06,1.00,06059,0.15,"472,384.00",...,medium,0,1,0,0,0,1,0,1,0
4,"-118,393309","34,227422",0.20,0.00,0.08,0.07,0.00,06037,0.55,"58,693.00",...,small,1,0,0,1,0,0,1,0,0


In [14]:
s_train, s_validate, s_test = prepare.split(scaled_df)

In [15]:
pd.set_option('display.max_columns', None)

In [16]:
cols = ['square_feet', 'lot_size', 'has_pool', 'age', 'county_Los Angeles', 'county_Orange', 'bedrooms', 'bathrooms']

X_train = s_train[cols]
y_train = s_train.assessed_value

X_validate = s_validate[cols]
y_validate = s_validate.assessed_value

X_test = s_test[cols]
y_test = s_test.assessed_value

In [17]:
train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [84]:
final_train_predictions = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
final_validate_predictions = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 
final_test_predictions = pd.DataFrame({
    'actual': s_test.assessed_value
})

## Multiple  Regression + RFE

In [47]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
k = 6

# 1. Transform our X
rfe = RFE(lm, n_features_to_select=6)
rfe.fit(X_train, y_train)
print('selected top 8 features:', X_train.columns[rfe.support_])
X_train_rfe = rfe.transform(X_train)

selected top 8 features: Index(['square_feet', 'lot_size', 'has_pool', 'county_Los Angeles', 'bedrooms',
       'bathrooms'],
      dtype='object')


### Looping through k

In [19]:
for k in range(2,8):
    lm = LinearRegression()
    # 1. Transform our X
    rfe = RFE(lm, n_features_to_select=k)
    rfe.fit(X_train, y_train)
    # 2. Use the transformed x in our model
    X_train_rfe = rfe.transform(X_train)
    X_validate_rfe = rfe.transform(X_validate)
    lm.fit(X_train_rfe, y_train)
    
    # Make predictions
    train_predictions['multiple_rfe_k=', k] = lm.predict(X_train_rfe)
    validate_predictions['multiple_rfe_k=', k] = lm.predict(X_validate_rfe)
    k+=1

In [20]:
train_predictions.head()

,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)"
12296,"312,195.00","336,926.27","304,208.06","302,288.82","340,423.06","333,856.22","335,418.30"
15891,"468,104.00","657,805.68","642,285.69","647,422.88","610,245.49","592,285.05","588,586.23"
35944,"415,247.00","276,661.57","268,318.86","253,749.24","238,723.59","235,782.03","233,499.44"
43195,"435,545.00","290,693.84","284,262.89","292,506.36","332,139.11","329,150.38","328,363.98"
19130,"87,774.00","287,226.96","315,060.54","303,365.03","290,085.33","287,845.56","294,278.69"


In [21]:
train_predictions['baseline'] = y_train.mean()

In [22]:
def calculate_rmse(y_predicted):
    return mean_squared_error(train_predictions.actual, y_predicted, squared = False)

train_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 7)   209,435.57
(multiple_rfe_k=, 6)   209,464.70
(multiple_rfe_k=, 5)   209,929.72
(multiple_rfe_k=, 4)   211,671.46
(multiple_rfe_k=, 3)   211,965.62
(multiple_rfe_k=, 2)   213,212.12
baseline               241,278.42
dtype: float64

In [23]:
validate_predictions['baseline'] = y_validate.mean()

In [24]:
validate_predictions.head()


,actual,"(multiple_rfe_k=, 2)","(multiple_rfe_k=, 3)","(multiple_rfe_k=, 4)","(multiple_rfe_k=, 5)","(multiple_rfe_k=, 6)","(multiple_rfe_k=, 7)",baseline
9695,"409,034.00","230,864.67","249,667.25","244,430.52","233,151.28","232,372.35","231,569.69","381,705.28"
14622,"651,348.00","485,591.30","476,096.95","481,820.20","510,815.77","500,822.66","496,399.76","381,705.28"
47991,"312,139.00","399,467.23","375,799.52","379,717.71","411,771.40","404,994.26","401,297.98","381,705.28"
8932,"91,589.00","239,270.75","259,343.04","277,307.43","263,360.00","261,864.30","254,945.59","381,705.28"
29805,"120,280.00","360,687.14","368,512.18","365,726.12","353,334.85","380,507.05","381,606.78","381,705.28"


In [25]:
def calculate_rmse(y_predicted):
    return mean_squared_error(validate_predictions.actual, y_predicted, squared = False)

validate_predictions.apply(calculate_rmse).sort_values()

actual                       0.00
(multiple_rfe_k=, 6)   209,698.04
(multiple_rfe_k=, 7)   209,713.40
(multiple_rfe_k=, 5)   210,002.87
(multiple_rfe_k=, 4)   212,225.99
(multiple_rfe_k=, 3)   212,648.76
(multiple_rfe_k=, 2)   213,620.47
baseline               238,353.52
dtype: float64

In [1]:
# k=7 is the best

228490-227162

1328

**K=7 has the lowest rmse, highest performance**

In [26]:
# Adding k=7 to final table
lm = LinearRegression()
 # 1. Transform our X
    
rfe = RFE(lm, n_features_to_select=7)
rfe.fit(X_train, y_train)
# 2. Use the transformed x in our model
X_train_rfe = rfe.transform(X_train)
X_validate_rfe = rfe.transform(X_validate)
X_test_rfe = rfe.transform(X_test)
lm.fit(X_train_rfe, y_train)
    
# Make predictions
final_train_predictions['multiple_rfe_k=7'] = lm.predict(X_train_rfe)
final_validate_predictions['multiple_rfe_k=7'] = lm.predict(X_validate_rfe)
final_test_predictions['multiple_rfe_k=7']=lm.predict(X_test_rfe)

NameError: name 'final_train_predictions' is not defined

## Polynomial Features

In [26]:
train_pred = pd.DataFrame({
    'actual': s_train.assessed_value
}) 
validate_pred = pd.DataFrame({
    'actual': s_validate.assessed_value
}) 

In [27]:
from sklearn.preprocessing import PolynomialFeatures

# 1. Generate Polynomial Features, k=2
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [28]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 2'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2
12296,"312,195.00","382,793.76","319,076.55"
15891,"468,104.00","382,793.76","587,729.19"
35944,"415,247.00","382,793.76","236,436.07"
43195,"435,545.00","382,793.76","324,396.88"
19130,"87,774.00","382,793.76","370,423.44"


In [29]:
validate_pred.head()

,actual,baseline,polynomial degree 2
9695,"409,034.00","381,705.28","242,689.39"
14622,"651,348.00","381,705.28","561,333.18"
47991,"312,139.00","381,705.28","485,694.73"
8932,"91,589.00","381,705.28","158,113.06"
29805,"120,280.00","381,705.28","411,849.42"


In [30]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2'], squared = False)

In [31]:
train_rmse_b = mean_squared_error(train_pred.actual,train_pred['baseline'], squared = False)
validate_rmse_b = mean_squared_error(validate_pred.actual,validate_pred['baseline'], squared = False)

In [32]:
train_rmse, validate_rmse

(203503.35735250247, 203562.76427455657)

In [33]:
validate_rmse-train_rmse

59.406922054098686

In [34]:
train_rmse_b, validate_rmse_b

(241278.41687336648, 238353.5244766119)

In [35]:
# Interaction terms only

poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)

X_validate_poly = poly.transform(X_validate)
train_pred['polynomial degree 2 only interaction'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 2 only interaction'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction
12296,"312,195.00","382,793.76","319,076.55","324,733.33"
15891,"468,104.00","382,793.76","587,729.19","587,189.50"
35944,"415,247.00","382,793.76","236,436.07","226,640.74"
43195,"435,545.00","382,793.76","324,396.88","329,935.08"
19130,"87,774.00","382,793.76","370,423.44","376,580.54"


In [36]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 2 only interaction'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 2 only interaction'], squared = False)
train_rmse, validate_rmse

(203729.70656880626, 203809.65022936126)

In [37]:
validate_rmse-train_rmse

79.94366055499995

In [38]:
# k=3
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [39]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 3'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 3'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3
12296,"312,195.00","382,793.76","319,076.55","324,733.33","308,778.11"
15891,"468,104.00","382,793.76","587,729.19","587,189.50","555,691.16"
35944,"415,247.00","382,793.76","236,436.07","226,640.74","244,421.69"
43195,"435,545.00","382,793.76","324,396.88","329,935.08","315,351.57"
19130,"87,774.00","382,793.76","370,423.44","376,580.54","311,337.67"


In [40]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 3'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 3'], squared = False)
train_rmse, validate_rmse

(201678.07158130204, 202821.9322561396)

In [41]:
# k=4
poly = PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [42]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 4'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 4'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4
12296,"312,195.00","382,793.76","319,076.55","324,733.33","308,778.11","313,475.12"
15891,"468,104.00","382,793.76","587,729.19","587,189.50","555,691.16","539,370.33"
35944,"415,247.00","382,793.76","236,436.07","226,640.74","244,421.69","237,501.60"
43195,"435,545.00","382,793.76","324,396.88","329,935.08","315,351.57","319,187.51"
19130,"87,774.00","382,793.76","370,423.44","376,580.54","311,337.67","296,429.99"


In [43]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(200148.06059221626, 203379.43563270045)

In [44]:
# k=5
poly = PolynomialFeatures(degree=5, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)

In [45]:
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
train_pred['baseline'] = y_train.mean()
validate_pred['baseline'] = y_validate.mean()
train_pred['polynomial degree 5'] = lm.predict(X_train_poly)
validate_pred['polynomial degree 5'] = lm.predict(X_validate_poly)
train_pred.head()

,actual,baseline,polynomial degree 2,polynomial degree 2 only interaction,polynomial degree 3,polynomial degree 4,polynomial degree 5
12296,"312,195.00","382,793.76","319,076.55","324,733.33","308,778.11","313,475.12","308,299.51"
15891,"468,104.00","382,793.76","587,729.19","587,189.50","555,691.16","539,370.33","537,965.97"
35944,"415,247.00","382,793.76","236,436.07","226,640.74","244,421.69","237,501.60","236,969.29"
43195,"435,545.00","382,793.76","324,396.88","329,935.08","315,351.57","319,187.51","332,474.52"
19130,"87,774.00","382,793.76","370,423.44","376,580.54","311,337.67","296,429.99","312,691.36"


In [46]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['polynomial degree 4'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['polynomial degree 4'], squared = False)
train_rmse, validate_rmse

(200148.06059221626, 203379.43563270045)

**K=3 has the lowest rmse**

In [88]:
# Adding k=3 to final table
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False)
poly.fit(X_train)
X_train_poly = pd.DataFrame(
    poly.transform(X_train),
    columns=poly.get_feature_names(X_train.columns),
    index=X_train.index,
)
lm = LinearRegression()
lm.fit(X_train_poly, y_train)
X_validate_poly = poly.transform(X_validate)
X_test_poly = poly.transform(X_test)
final_train_predictions['baseline'] = y_train.mean()
final_validate_predictions['baseline'] = y_validate.mean()
final_test_predictions['baseline'] = y_test.mean()
final_train_predictions['polynomial degree 3'] = lm.predict(X_train_poly)
final_validate_predictions['polynomial degree 3'] = lm.predict(X_validate_poly)
final_test_predictions['polynomial degree 3'] = lm.predict(X_test_poly)

## Lasso-Lars

In [46]:
from sklearn.linear_model import LassoLars
# create the model object
lars = LassoLars(alpha=0)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)

pd.Series(lars.coef_, index=X_train.columns).sort_values()

bedrooms             -254,508.76
lot_size             -176,477.70
county_Los Angeles    -53,328.55
county_Orange          12,474.60
has_pool               37,162.21
age                    84,325.44
bathrooms             198,331.49
square_feet           961,064.22
dtype: float64

In [47]:
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['lasso_lars'] = X_train_pred_lars
validate_pred['lasso_lars'] = X_validate_pred_lars

In [48]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['lasso_lars'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['lasso_lars'], squared = False)
train_rmse, validate_rmse

(229427.37534789668, 227596.58856245351)

In [49]:
validate_rmse-train_rmse

-1830.786785443168

In [90]:
# Adding to final table
# create the model object
lars = LassoLars(alpha=1)

# fit the model to our training data
lars.fit(X_train, y_train)

# predict validate
X_train_pred_lars = lars.predict(X_train)
X_validate_pred_lars = lars.predict(X_validate)
X_test_pred_lars = lars.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['lasso_lars'] = X_train_pred_lars
final_validate_predictions['lasso_lars'] = X_validate_pred_lars
final_test_predictions['lasso_lars'] = X_test_pred_lars

## Generalized Linear Model

In [50]:
from sklearn.linear_model import TweedieRegressor

# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
# Add lassolars predictions to our predictions DataFrame
train_pred['glm'] = X_train_predict_glm
validate_pred['glm'] = X_validate_predict_glm

In [51]:
train_rmse = mean_squared_error(train_pred.actual,train_pred['glm'], squared = False)
validate_rmse = mean_squared_error(validate_pred.actual,validate_pred['glm'], squared = False)
train_rmse, validate_rmse

(230888.78077339692, 230002.83186807355)

In [52]:
validate_rmse-train_rmse

-885.9489053233701

In [91]:
# Adding to final table

glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data
glm.fit(X_train, y_train)

# predict train
X_train_predict_glm = glm.predict(X_train)
X_validate_predict_glm = glm.predict(X_validate)
X_test_predict_glm = glm.predict(X_test)
# Add lassolars predictions to our predictions DataFrame
final_train_predictions['glm'] = X_train_predict_glm
final_validate_predictions['glm'] = X_validate_predict_glm
final_test_predictions['glm'] = X_test_predict_glm

## Evaluation

In [94]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_train_predictions.actual, y_predicted, squared = False)

final_train_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   223,633.46
lasso_lars            232,745.23
multiple_rfe_k=7      232,763.09
glm                   235,321.90
baseline              274,862.72
dtype: float64

In [95]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_validate_predictions.actual, y_predicted, squared = False)

final_validate_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   222,070.92
lasso_lars            230,203.61
multiple_rfe_k=7      230,221.87
glm                   233,617.59
baseline              272,468.83
dtype: float64

In [96]:
def calculate_rmse(y_predicted):
    return mean_squared_error(final_test_predictions.actual, y_predicted, squared = False)

final_test_predictions.apply(calculate_rmse).sort_values()

actual                      0.00
polynomial degree 3   225,323.49
lasso_lars            233,614.86
multiple_rfe_k=7      233,644.57
glm                   237,682.56
baseline              277,387.85
dtype: float64

In [97]:
(277387.85-225323.49)/277387.85

0.18769517121964782